In [ ]:
import torch
from PIL import Image
from model_zoo import get_model   # 你脚本里定义的
import argparse

# -----------------------
# 参数设定
# -----------------------
device = "cuda:3" if torch.cuda.is_available() else "cpu"
model_name = "llava1.5"    # 可选 llava1.5 / llava1.6
method = "adapt_vis"         # 可选 "default" / "scaling_vis" / "adapt_vis"

# -----------------------
# 加载模型
# -----------------------
# get_model 返回 (model, image_preprocess)
model, image_preprocess = get_model(model_name, device, method)

# -----------------------
# 输入数据
# -----------------------
image_path = "test.jpg"
question = "User: <image>\n What is the color of the girl's hair ?\nAssistant:"

# 预处理图像（根据模型返回的 image_preprocess）
image = Image.open(image_path).convert("RGB")


# -----------------------
# 构造输入
# -----------------------
inputs = model.processor(
    text=question,
    images=image,
    return_tensors="pt",
    padding="max_length",
    max_length=77
).to(device)

# 确保 eos 和 pad 不为 None
inputs["eos_token_id"] = model.tokenizer.eos_token_id
inputs["pad_token_id"] = model.tokenizer.pad_token_id

# -----------------------
# 推理
# -----------------------
with torch.no_grad():
    output = model.model.generate(
        **inputs,
        weight=1.0,  # 如果你想调整权重，可以修改这里   
        max_new_tokens=50,
        do_sample=False,
        return_dict_in_generate=True,
        output_scores=True
    )

# -----------------------
# 解码答案
# -----------------------
answer = model.processor.decode(
    output["sequences"][0][len(inputs["input_ids"][0]):],
    skip_special_tokens=True
)

print(f"Q: {question}")
print(f"A: {answer}")


/home/tuo/anaconda3/envs/adaptvis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
LlavaForConditionalGenerationScal has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an ex

TypeError: unsupported operand type(s) for *=: 'Tensor' and 'NoneType'

In [ ]:
import torch
from PIL import Image
from model_zoo import get_model
from model_zoo.llava15 import change_greedy_to_add_weight   # 导入替换函数

# -----------------------
# 参数设定
# -----------------------
device = "cuda:3" if torch.cuda.is_available() else "cpu"
model_name = "llava1.5"    
method = "adapt_vis"

# -----------------------
# 替换 generate 的 greedy_search
# -----------------------
change_greedy_to_add_weight()

# -----------------------
# 加载模型
# -----------------------
model, image_preprocess = get_model(model_name, device, method)

# -----------------------
# 输入数据
# -----------------------
image_path = "test.jpg"
question = "User: <image>\n What is the color of the girl's hair ?\nAssistant:"

image = Image.open(image_path).convert("RGB")
if image_preprocess is not None:
    image = image_preprocess(image)

inputs = model.processor(
    text=question,
    images=image,
    return_tensors="pt",
    padding="max_length",
    max_length=77
).to(device)

# -----------------------
# adapt_vis 逻辑
# -----------------------
@torch.no_grad()
def adapt_vis_generate(model, inputs, weight1=0.5, weight2=1.5, threshold=0.7):
    # baseline
    output = model.model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=False,
        return_dict_in_generate=True,
        output_scores=True,
        weight=1.0,        # baseline 一定要有 weight
    )

    # uncertainty
    first_step_logits = output["scores"][0]
    uncertainty = float(torch.nn.functional.softmax(first_step_logits, dim=-1)[0].max().cpu())
    print(f"[Uncertainty] {uncertainty:.3f}")

    # 挑权重
    weight = weight1 if uncertainty < threshold else weight2

    # 再跑一次
    output = model.model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=False,
        return_dict_in_generate=True,
        output_scores=True,
        weight=weight
    )
    return output

# -----------------------
# 推理
# -----------------------
output = adapt_vis_generate(model, inputs, weight1=0.5, weight2=1.5, threshold=0.7)

answer = model.processor.decode(
    output["sequences"][0][len(inputs["input_ids"][0]):],
    skip_special_tokens=True
)

print(f"Q: {question}")
print(f"A: {answer}")


/home/tuo/anaconda3/envs/adaptvis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
LlavaForConditionalGenerationScal has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an ex